# Decorators

Consider another version of the `make_polite` function:

In [ ]:
def make_polite(func):
    """
    First, we define an 'inner()' function! Note: we call it 'inner' here, but it could be anything.
    This inner function prints 'Hello, world! ', then makes a call to the function being decorated, passed as argument.
    Finally, the decorator returns the 'inner()' function.
    """
    
    def inner(*args, **kwargs):    # We create another function within the decorator!    
        print("Hello, world! ",)   # This is a decoration, the added functionality which will happen everytime the decorated function is called.
        func(*args, **kwargs)      # Call to the function, passed as argument, that is decorated 
        
    return inner                   # We return the reference to the inner function that we have created.                                                                                  


# Let's define a few functions
def one_function():                                                                                                 
    print("I am function one!")      
    
def another_function():                                                                                                 
    print("I am function two!")                                                                                          


# Regular calls to the two functions above
print("# We can call our functions as we would \
normally do (without decoration):")
one_function()
another_function()

# If we want to run a function with a decoration, we need to pass the 
# function to the decorator, as an argument: 
make_polite(one_function)
# However, this ^^^^^^^ is not enough to *run* the decorated function 'one_function'
# Note that this does not print our messages!


#### HW-1:

Consider writing a similar decorator, but instead of printing 
`Hello, world! `, you would print  `Greetings from <your_name>` plus 
the time of the day before calling the function. 
Write that decorator below, calling it `make_personal_greeting`.
Test it on `another_function` (see lecture notebook; repeated below). 

In [ ]:
from datetime import datetime

current_time = datetime.now().time().strftime("%H:%M")

def make_personal_greeting(func):
    """
    First, we define an 'inner()' function! Note: we call it 'inner' here, but it could be anything.
    This inner function now prints 'Greetings from <name>' and the time of the day before calling the function.
    """
    
    def inner(*args, **kwargs):                               # We create another function within the decorator!    
        print("Greetings from ", *args, "at", current_time)   # This is a decoration, the added functionality which will happen everytime the decorated function is called.
        func(*args, **kwargs)                                 # Call to the function, passed as argument, that is decorated 
        
    return inner                   # We return the reference to the inner function that we have created.       

print(make_personal_greeting("Clark Kent"))  # => Greetings from Clark Kent at 13:45
                                             # => another_function() from FA-2 decorated with personal greeting
# Now the decoration
@make_personal_greeting
def another_function(name):                                                                                                 
    print("Madam! I am function two!")

another_function("Diana Prince")  # => Greetings from Diana Prince at 13:45

Now, use both decorators:

In [ ]:
@make_polite
@make_personal_greeting
def another_function(name):
    print("Madam! I am function two!")

another_function("Ariana Grande")

###  Passing arguments to a **decorated function**

As you saw above, we may want to pass arguments to decorated functions. The code below implements a more extensive decorator that can be applied to functions receiving any number of arguments. We accept _*args_ and _**kwargs_ in the inner function to cover for both positional and named arguments.

Try to understand what this code does. All the elements identified in class, in the description of a decorator's anatomy can be found here as well. Try to match them, and see what has been adjusted to decorate functions with arguments. 

**Note:** the function `blub()` is not very meaningful. It is just a mock simulation of a function that may occasionally cause exceptions (which has been implemented simply as a random draw).

**Note 2:** If you are not familiar with exceptions, you can read here: https://docs.python.org/3/tutorial/errors.html

In [ ]:
from random import randint

# Let's define another decorator, 
# which is able to decorate a function (fun) that receives arguments:
def with_retry(func):
    def retried(*args, **kwargs):
        for _ in range(5): # hardcoded range
            try:
                func(*args, **kwargs)
            except Exception as e:
                print("Caught exception! {}".format(e))
                print("The exception was raised while calling {} \
                      with args: {}, kwargs: {}. Retrying".format(func, args, kwargs))
    return retried

@with_retry
def blub(m):
    rnd = randint(0,m)
    print(rnd)
    if rnd < m - 2:
        raise Exception("\nSome exception occurred ...")
        
# You can also decorate blub() without using pie-decorator syntax:
# blub = with_retry(blub)  

blub(7)


###  Passing arguments to a **decorator**

In the example above, the decorator *with_retry()* has a hardcoded value (the number of trials, which is set to 5). This is not great practice: we would have to write another decorator if, for instance, we need 10 trials instead!

To fix this, we can write a decorator which recieves the number of trials as an argument!

See how to do this in the example below:

In [ ]:
from random import randint

def with_retry(trials):      # The decorator now has an argument!
    def wrap(func):          # Note this additional wrapping layer!!!
        def retried(*args, **kwargs):
            for _ in range(trials):
                try:
                    func(*args, **kwargs)
                except Exception as e:
                    print("Caught exception! {}".format(e))
                    print("The exception was raised while calling {} \
                      with args: {}, kwargs: {}. Retrying".format(func, args, kwargs))
        return retried
    return wrap

@with_retry(10)
def blub(m):
    rnd = randint(0,m)
    print(rnd)
    if rnd < m - 2:
        raise Exception("Some exception occurred ...")
        
# Alternative syntax
# blub = with_retry(10)(blub)  

blub(7)


#### HW-2

Write a decorator named `myDecorator` that takes two functions as arguments.
The first function should be called **before** calling the decorated function, while the second function should be called **after** the decorated function. 

Write also the code to create a decorated function, such that it produces the output specified below.


In [ ]:
## Your solution to HW-2 goes here:
def myDecorator(before, after):
    def wrap(func):
        def decorated_function(*args, **kwargs):
            before()
            func(*args, **kwargs)
            after()
        return decorated_function
    return wrap


###########################

# Example:
@myDecorator(lambda: print("before"), lambda: print("after"))
def decorated_function(message):
    print("main function: " + message)

# Alternative syntax:
# decorated_function = myDecorator(lambda: print("before"), lambda: print("after"))(decorated_function)

###########################

decorated_function("hey") # => prints:  before
                          #             main function: hey
                          #             after


#### HW-3:

Write three decorators named `make_underline`, `make_bold`, and `make_italic` that format text, in a way that we receive the expected outputs. 

**Note:** There are corresponding HTML tags for `underline (<u>  </u>)`, `bold
(<b>  </b>)`, and `italic (<i>  </i>)`, respectively and we are trying to implement those in text by decorators.  


In [ ]:
## Your solution to HW-3 goes here:
def make_bold(func):
    def wrapper(*args, **kwargs):
        text = func(*args, **kwargs)
        return f"<b>{text}</b>"
    return wrapper


def make_italic(func):
    def wrapper(*args, **kwargs):
        text = func(*args, **kwargs)
        return f"<i>{text}</i>"
    return wrapper


def make_underline(func):
    def wrapper(*args, **kwargs):
        text = func(*args, **kwargs)
        return f"<u>{text}</u>"
    return wrapper


###########################

@make_underline
def hello():
    return "Hello, world!"
print(hello()) ## returns "<u>Hello, world!</u>"

@make_bold
def hello():
    return "Hello, world!"
print(hello()) ## returns "<b>Hello, world!</b>"

@make_italic
def hello():
    return "Hello, world!"
print(hello()) ## returns "<i>Hello, world!</i>"


### HW-3.1:
Now, make the text bold and underlined:

In [ ]:
@make_bold
@make_underline
def hello():
    return "Hello, world!"
print(hello())


#### HW-3.2:
Write a function called `hey_there()` that returns the string: "Hey there from me!".
Decorate it so that it is both italic and underlined. 
Use your previously defined `make_italic` and `make_underline` decorators. 
Store the final decorated result in the variable `result_hey_there`.

In [ ]:
## Your solution to HW-3.2 goes here:
@make_underline
@make_italic
def hey_there():
    return "Hey there from me!"

###########################

print(hey_there()) # => <u><i>Hey there from me!</i></u>


---

## Currying

Recall that currying is the process of turning **multi-argument** functions into a **composition of single-argument** functions.


Now consider that we want to apply any three of the above style modifications. We can write the following functions

In [ ]:
def make_bold(text):
    return f"<b>{text}</b>"
def make_italic(text):
    return f"<i>{text}</i>"
def make_underline(text):
    return f"<u>{text}</u>" 

def beeautify_font(text, style1, style2, style3):
    return style1(style2(style3(text)))

N = beeautify_font("This is fancy text!", make_bold, make_italic, make_underline)
print(N)  # => <b><i><u>This is fancy text!</u></i></b>

### HW-4:
Create a curried version of the above function.

In [ ]:
## Your solution to HW-3.2 goes here:
def beeautify_font_c(style1):
        def apply_style2(style2):
            def apply_style3(style3):
                def apply_text(text):
                    return style1(style2(style3(text)))
                return apply_text
            return apply_style3
        return apply_style2

###########################

result_hey_there = beeautify_font_c(make_underline)(make_italic)(make_bold)
print(result_hey_there("I love prolog!")) # => <u><i><b>I love prolog!</b></i></u>

### HW-4.1
Let's now call it in several ways and see which one is correct and will print "I love prolog" with any (or no) style applied and which one is not (leading to an error or not printing "I love prolog!" in any form):

In [ ]:
# 1:
result_hey_there_1 = beeautify_font_c(make_underline)(make_italic)(make_bold)
# print(result_hey_there_1("I love prolog!")) 

# 2: 
result_hey_there_2 = beeautify_font_c(make_underline)(make_bold)
# print(result_hey_there_2("I love prolog!")) 

# 3: 
# print(result_hey_there_2(make_bold)("I love prolog!")) 

# 4:
# print(result_hey_there_2("I love prolog!")(make_bold)) 

# 5:
# print(result_hey_there_2("I love prolog!")()) 

# 6:
# print(result_hey_there_2()("I love prolog!")) 

# 7:
# print(result_hey_there_2("make_bold")("I love prolog!")) 



In [ ]:
# Non-curried function:
f = lambda x,y,z: (x + y) * z

# Curried function 
def fcurry(x, *args):
    def f1(y, *args):
        def f2(z):
            return (x + y) * z
        if args:
            return f2(*args)
        return f2
    if args:
        return f1(*args)
    return f1

print("Non-curried vs. manual curried example:")
print(f(1,2,3))
print(fcurry(1,2,3))


The function above is correct, but it is very convoluted and difficult to read, which makes it impractical and prone to errors. 

But currying a function is like adding some flavor to it ;), so a natural way to implement currying is to use decorators to add curry to our functions. The `PyMonad` library allows us to do so.

In [ ]:
%pip install PyMonad
# (uncomment the above^ line, the first time you run this code)

from pymonad.tools import curry

@curry(3)     # 3 is the number of arguments we want to curry
def f(x,y,z):
    return (x+y)*z

# We can call the function as we would normally do to obtain the final result:
print(f(10,20,30))


In [ ]:
# We can also access the intermediate functions. 
# Since we curried 3 arguments, 
# we get two intermediate functions before obtaining the final result.

# If we call our curried function f with one argument, 
# it will return the first intermediate function "specialised" on that argument. 
x = 10
f1 = f(x)
print(f1)

# Can you think what does f1 do? What should the following line return?
f1(4,3)


In [ ]:
# Now we can call the intermediate specialised function f1 with another argument.
# In this way, we can access the second intermediate function, 
# which will be specialised on x and y
y = 20
f2 = f1(y)
print(f2)

# f2 is now specialised running the operation with x=1 and y=2.
# We can call it with a third argument and get the final result:
z = 30
print(f2(z))

# Check this is the same as calling f with all three arguments
print(f(x,y,z) == f2(z))

# We can also access the second intermediate function directly 
# providing 2 arguments to f:
f2 = f(x)(y)
print(f2)
print(f2(z))


In [ ]:
@curry(4)
def systolic_bp(bmi, age, sex_male, treatment):
    return 68.15 + (0.58 * bmi) + (0.65 * age) + (0.94 * sex_male) + (6.44 * treatment)

# Example: 
systolic_bp(25, 50, 1, 0)


Since we have curried the function, we can look at the intermediate steps, and create **specialised** functions. Let's say we want to have a function tailored to a patient (*i.e.*, with their BMI, age and sex pre-defined) which we can use to predict blood pressure before and after a treatment:

In [ ]:
bp_patient = systolic_bp(25, 50, 0)

treated = 0
print("Before treatment:{:>9}".format(bp_patient(treated)))

treated = 1
print("After treatment:{:>10}".format(bp_patient(treated)))


#### HW-4.2
A research project is investigating the effect of the BMI and age on the blood pressure, for women who have been previously treated. It would be useful to have a specialised function for that group, such that its only arguments are the BMI and age of the patient. Can you write this specialised function named *bp_treated_women* based on the curried function above? What do you need to change?

**Note:** do not forget to write the test of your function to prove it works!

In [ ]:
## Your solution to HW-4.2 goes here:
@curry(4)
def systolic_bp(sex_male, treatment, bmi, age):
    return 68.15 + (0.58 * bmi) + (0.65 * age) + (0.94 * sex_male) + (6.44 * treatment)

bp_patient_women = systolic_bp(0)(1)

###########################

bp_patient_women(25, 50)


---

## Partial

The `functools` module in Python also gives us the option to work with Partial functions. 

Partial functions are specialised functions, **bound to certain values**. This makes the Partial design pattern very similar to Currying, to the extent that these two are often confused, so be mindful of this when you search information online! 

The main difference is that the goal of Currying is to create functions that accept one single argument, while there is no such requirement in Partial Functions. This also results in differences in how they are used. 

See an example for the specialised `add()` function below:

In [ ]:
from functools import partial
from operator import add

# The following code creates a function to increment in 1.
# It uses composition of single-argument functions (add1, add).
# add1 is a specialised case of add
def add1(x):
    return add(1, x)

print(add1(2)) # => 3

# Let's implement add1 using Partial
add1 = partial(add, 1) # We provide the first argument to obtain a specialised function
print(add1)    # add1 is a partial function that always increments in 1
print(add1(2)) # Returns a value (3)


Note the difference when doing the same with currying:

In [ ]:
curried_add = curry(2,add) # We need to specify that we curry 2 arguments
                           # We still don't provide values
print(curried_add)
add1 = curried_add(1)      # We need this extra line to get a specialised function
print(add1(2))             # Returns a value (3)


Another example:

In [ ]:
from functools import partial

def operation(x, y, z, message):
        return message + ' ' + str(x * (y + z))

# Let's create a function that multiplies by 2
doublesum = partial(operation, 2) # operation specialised in x = 2
doublesum(3,1, "Result:")         # Result: 8


#### HW-5:

#### HW-5.1:
Turn this example into a specialised function in which x=3 and y=5.<br>
Try also specializing on x=3 and y=5 and z=7.  Name the functions `specialised1` and `specialised2`, respectively.

In [ ]:
## Your solution to FA-16 (part I) goes here:
x = 3
y = 5
z = 7

specialised1 = partial(operation, x, y)          
specialised2 = partial(operation, x, y, z)

###########################


print(specialised1(7, "Specialised in x=%i and y=%i. Result:"%(x,y))) # Specialised in x=3 and y=5: Result: 36

print(specialised2("With x=%i and y=%i and z=%i:"%(x,y,z)))           # With x=3 and y=5 and z=7: 36



#### HW-5.2: 
Would you be able to specialise the function only on z? Name it *specialised3*.

In [ ]:
def operation_z(x, y, z, message):
        return message + ' ' + str(x * (y + z))

## Your solution to HW-5.2 goes here:

specialised3 = partial(operation, z=z)

###########################

print(specialised3(x=x, y=y ,message="Specialised in z=%i. Result:"%(z))) # => Specialised in z=7. Result: 36


#### HW-5.3:

Create the specialised function *specialised2* with currying instead. 
What is the main difference?

In [ ]:
## Your solution to HW-5.3 goes here:

specialised2_currying = curry(3)(operation)(x)(y)

###########################

print(specialised2_currying(z,"With x=%i and y=%i and z=%i:"%(x,y,z))) # With x=3 and y=5 and z=7: 36


---

# Closures

Finally, another way to have specialised functions is the use of Closures.

A closure is a function that has **bound** variables, *i.e.*, it stores some variables and their values in a referencing environment. A function with **a closure can reference a variable that was available in the scope where the function was originally defined, but which would normally not be available in the scope where it is executed**.

Check the example below. For simplicity, the  `inner()` function returned by `pop_up()` only prints a text message through the standard output, but this example would be equivalent if `inner()` incorporated a more complex way to visualise the message (*e.g.*, launching a pop-up window).

In [ ]:
def pop_up(message):    
    def inner():
        # When we define this function, message is in the scope of the pop_up() function
        print(message)      
    # When returning the inner function, the message variable (and its value!) will be 
    # provided together with inner, as a closure (!)
    return inner 


# We can create specialised functions using the closure
greeter = pop_up('Hello!')

# We can now use the function without providing any argument:
greeter()

# The code above works because the message was stored in an attribute of the function called __closure__:
print("Content of the closure:", greeter.__closure__[0].cell_contents)

# We can create other specialised functions
checkpoint = pop_up('Passing by')
error = pop_up('Error!')

# And we can use them as many times as we want elsewhere in our code
greeter()
checkpoint()
error()
checkpoint()


# Compare this to a version without closures:
def no_closure_pop_up(message):
    print(message)
    
no_closure_pop_up('Error!')

no_closure_pop_up('Hello!')

no_closure_pop_up('Error!') # We need to provide the argument each time!

#### HW-6:

The function defined below applies a tax rate to an amount (price).

In [ ]:
def taxer(amount, rate):
    return amount + (amount * (float(rate)) / 100 )

price1 = 500
# Apply taxes:
price1 = taxer(price1, 19)

price2 = 79
# Apply same tax:
price2 = taxer(price2, 19)

price3 = 30
# This one requires another tax
price3 = taxer(price3, 7)


#### HW 6.1:
Implement and test a variant that uses closures. Name it `taxer_closure`. Which variable(s) will you add to the closure?

In [ ]:
## Your solution to HW-6.1 goes here:
def taxer_closure(rate):
    def apply_tax(amount):
        def compute_tax():
            return amount + (amount * (float(rate)) / 100 )
    return apply_tax

###########################


## Final words:
The four design patterns that we have seen in this part of Module 1 are very related: they provide some form of function decorations or specialisations, and implement that with the use of higher-order functions or function composition. Beware of the differences between them and the nuances in their implementation!

## References:
* Notebook adapted and extended from Martin Atzmueller, Materials for Advanced Programming for CSAI (2019) and Fred Blain's notebook from Materials for AP4CSAI (2025).